In [29]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import os
os.environ['KERAS_BACKEND']='tensorflow' # set backend

In [30]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras.datasets import imdb
from keras import backend as K

Using TensorFlow backend.


In [3]:
# set parameters:
max_features = 5000
maxlen = 100

In [1]:
# look into preprocessor.py file for details
from preprocessor import Index_Sequence, accuracy, indicator_to_matrix

In [4]:
yelp_data = Index_Sequence("yelp_academic_dataset_review.json", "text", "stars", 500,maxlen)

In [96]:
embedding_dims = 50
nb_filter = 250
filter_length = 3
hidden_dims = 250

nb_classes = len(yelp_data.docs_label_index)

In [97]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use max pooling:
model.add(MaxPooling1D(pool_length=model.output_shape[1]))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

Build model...


In [98]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
model.summary()

NameError: name 'model' is not defined

In [99]:
#from preprocessor import Dataset, pad_vec_sequences, pad_sent_sequences
from sklearn import preprocessing
from sklearn import cross_validation

In [100]:
# data split
x_train, x_test, y_train, y_test = cross_validation.train_test_split(yelp_data.X_doc_seq,yelp_data.Y_doc_seq,test_size=0.2)

# create appropirate matrix (hot encoded) response
y_train, y_test = [indicator_to_matrix(x,yelp_data.docs_label_index)  for x in (y_train, y_test)]

In [103]:
# model specs
batch_size = 32
nb_epoch = 10

# train model
model.fit(x_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(x_test, y_test))

Train on 400 samples, validate on 100 samples
Epoch 1/10
400/400 [==============================] - 9s - loss: 0.7346 - acc: 0.8525 - val_loss: 1.4534 - val_acc: 0.3700
Epoch 2/10
400/400 [==============================] - 9s - loss: 0.5187 - acc: 0.9125 - val_loss: 1.5223 - val_acc: 0.3200
Epoch 3/10
400/400 [==============================] - 9s - loss: 0.3575 - acc: 0.9475 - val_loss: 1.5847 - val_acc: 0.3500
Epoch 4/10
400/400 [==============================] - 9s - loss: 0.2231 - acc: 0.9800 - val_loss: 1.8554 - val_acc: 0.2600
Epoch 5/10
400/400 [==============================] - 11s - loss: 0.1562 - acc: 0.9675 - val_loss: 1.9851 - val_acc: 0.3200
Epoch 6/10
400/400 [==============================] - 11s - loss: 0.0947 - acc: 0.9850 - val_loss: 2.0631 - val_acc: 0.3200
Epoch 7/10
400/400 [==============================] - 12s - loss: 0.0757 - acc: 0.9850 - val_loss: 2.0960 - val_acc: 0.3600
Epoch 8/10
400/400 [==============================] - 10s - loss: 0.0612 - acc: 0.9850 - v

In [104]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

100/100 [==============================] - 1s     
Test loss: 2.35596658707
Test accuracy: 0.32


In [ ]:
model.save('yelp_cnn_hidden_epocs.h5')

In [ ]:
from keras.models import load_model
current_model = load_model('yelp_cnn_hidden_epocs.h5')